# Jump process using Catalyst.jl
Simon Frost (@sdwfrost), 2020-05-11, updated 2021-03-15

## Introduction

This implementation defines the model as a combination of two jump processes, infection and recovery, simulated using the [Doob-Gillespie algorithm](https://en.wikipedia.org/wiki/Gillespie_algorithm).

## Libraries

In [ ]:
using Catalyst
using DiffEqJump
using Random
using DataFrames
using StatsPlots
using BenchmarkTools

## Transitions

In [ ]:
sir_model = @reaction_network begin
  β*c/(s+i+r), s + i --> 2i
  γ, i --> r
end β c γ;

## Time domain

In [ ]:
tmax = 40.0
tspan = (0.0,tmax);

For plotting, we also define a separate time series.

In [ ]:
δt = 0.1
t = 0:δt:tmax;

## Initial conditions

In [ ]:
u0 = [990,10,0]; # S,I,R

## Parameter values

In [ ]:
p = [0.05,10.0,0.25];

## Random number seed

We set a random number seed for reproducibility.

In [ ]:
Random.seed!(1234);

## Running the model

Running this model involves:

- Setting up the problem as a `DiscreteProblem`;
- Adding the jumps and setting the algorithm using `JumpProblem`; and
- Running the model, specifying `SSAStepper`.

In [ ]:
prob_discrete = DiscreteProblem(sir_model,u0,tspan,p);

In [ ]:
prob_jump = JumpProblem(sir_model,prob_discrete,Direct());

In [ ]:
sol_jump = solve(prob_jump,SSAStepper());

## Post-processing

In order to get output comparable across implementations, we output the model at a fixed set of times.

In [ ]:
out_jump = sol_jump(t);

We can convert to a dataframe for convenience.

In [ ]:
df_jump = DataFrame(out_jump')
df_jump[!,:t] = out_jump.t;

## Plotting

We can now plot the results.

In [ ]:
@df df_jump plot(:t,
    [:x1 :x2 :x3],
    label=["S" "I" "R"],
    xlabel="Time",
    ylabel="Number")

## Benchmarking

In [ ]:
@benchmark solve(prob_jump,SSAStepper())